# Initializing and Restoring EMI-RNN Graphs

The *EMI-RNN* implementation supports four forms of initialization/restoring:
1. An entirely new graph can be constructed with randomly initialized weights.
2. A saved graph can be loaded into the current `EMI_Driver`.
2. An entirely new graph can be constructed with weights initialized from a saved graph. This behavior is essentially restoration of a saved graph.
3. (*Experimental*) Initializing/Restoring using numpy matrices. 

All three methods will be illustrated in this notebook. This notebook uses the HAR dataset and builds on the [EMI LSTM example.ipynb](00_emi_lstm_example.ipynb)

Adapted from Microsoft's notebooks, available at https://github.com/microsoft/EdgeML authored by Dennis et al.

In [ ]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
# To include edgeml in python path
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

# MI-RNN and EMI-RNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_BasicLSTM
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

Let us set up some network parameters for the computation graph.

In [2]:
# Network parameters for our LSTM + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 64
NUM_FEATS = 16
FORGET_BIAS = 1.0
NUM_OUTPUT = 5
USE_DROPOUT = True
KEEP_PROB = 0.75

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 32

# Number of epochs in *one iteration*
NUM_EPOCHS = 3
# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4
# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 10

# A staging direcory to store models
dataset = 'DREAMER'
MODEL_PREFIX = '/home/sf/data/EdgeML/tf/examples/EMI-RNN' + dataset + '/model-lstm'

In [2]:
# Network parameters for our LSTM + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 350
NUM_FEATS = 8
FORGET_BIAS = 1.0
NUM_OUTPUT = 5
USE_DROPOUT = True
KEEP_PROB = 0.75

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 32

# Number of epochs in *one iteration*
NUM_EPOCHS = 3
# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4
# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 10

# A staging direcory to store models
dataset = 'WESAD'
MODEL_PREFIX = '/home/sf/data/EdgeML/tf/examples/EMI-RNN' + dataset + '/model-lstm'

# Loading Data

In [3]:
path="/home/sf/data/DREAMER/Dominance/64_16/"
x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (61735, 5, 64, 16)
y_train shape is: (61735, 5, 5)
x_test shape is: (6860, 5, 64, 16)
y_test shape is: (6860, 5, 5)


In [9]:
path="/home/sf/data/WESAD/350_116/"
#x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
#x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
#BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
#BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_test.shape[1]
#print("x_train shape is:", x_train.shape)
#print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_test.shape)
print("y_test shape is:", y_test.shape)

x_test shape is: (16863, 5, 350, 8)
y_test shape is: (16863, 5, 5)


In [10]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True
    
def addExtendedAssignOps(self, graph, W_val=None, B_val=None):
    W1 = graph.get_tensor_by_name('W1:0')
    B1 = graph.get_tensor_by_name('B1:0')
    W1_op = tf.assign(W1, W_val)
    B1_op = tf.assign(B1, B_val)
    self.assignOps.extend([W1_op, B1_op])

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob, **kwargs):
    feedDict = {self._emiGraph.keep_prob: keep_prob}
    return feedDict
    
EMI_BasicLSTM._createExtendedGraph = createExtendedGraph
EMI_BasicLSTM._restoreExtendedGraph = restoreExtendedGraph
EMI_BasicLSTM.addExtendedAssignOps = addExtendedAssignOps

if USE_DROPOUT is True:
    EMI_Driver.feedDictFunc = feedDictFunc

## 1. Initializing a New Computation Graph

In the most common use cases, a new EMI-RNN graph would be created and trained

In [11]:
tf.reset_default_graph()

inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiLSTM = EMI_BasicLSTM(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

# Construct the graph
g1 = tf.Graph()    
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)


W0809 11:12:32.864859 140177766897472 deprecation_wrapper.py:119] From /home/sf/data/EdgeML/tf/edgeml/graph/rnn.py:1141: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 11:12:32.881156 140177766897472 deprecation_wrapper.py:119] From /home/sf/data/EdgeML/tf/edgeml/graph/rnn.py:1153: The name tf.data.Iterator is deprecated. Please use tf.compat.v1.data.Iterator instead.

W0809 11:12:32.882301 140177766897472 deprecation.py:323] From /home/sf/data/EdgeML/tf/edgeml/graph/rnn.py:1153: DatasetV1.output_types (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
W0809 11:12:32.883116 140177766897472 deprecation.py:323] From /home/sf/data/EdgeML/tf/edgeml/graph/rnn.py:1154: DatasetV1.output_shapes (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use 

Lets initialize a new session with this graph and train a model. The saved model will be used later for restoring.

In [12]:
emiDriver.initializeSession(g1)

W0809 11:13:19.965286 140177766897472 deprecation_wrapper.py:119] From /home/sf/data/EdgeML/tf/edgeml/trainer/emirnnTrainer.py:369: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0809 11:13:19.982229 140177766897472 deprecation_wrapper.py:119] From /home/sf/data/EdgeML/tf/edgeml/trainer/emirnnTrainer.py:372: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



As the output above indicates, the last restored model is `/tmp/model-lstm-1001`. That is, with `MODEL_PREFIX = '/tmp/model-lstm'` and `GLOBAL_STEP=1001`.

In [13]:
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index

In [23]:
%%time

k = 1
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)

CPU times: user 4min 49s, sys: 23.3 s, total: 5min 13s
Wall time: 1min 1s


In [24]:
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

Accuracy at k = 1: 0.087825


In [26]:
type(x_test[:64])

numpy.ndarray

## 2. Loading a Saved Graph into EMI-Driver

We will reset the computation graph, load a saved graph into the current `EMI_Driver` and verify its outputs.

In [8]:
MODEL_PREFIX='/home/sf/data/EdgeML/tf/examples/EMI-RNN//DREAMER/model-lstm'

In [26]:
MODEL_PREFIX='/home/sf/data/EdgeML/tf/examples/EMI-RNN//WESAD/model-lstm'

In [27]:
tf.reset_default_graph()
emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX, 1007)
k = 2

W0809 11:18:00.678148 140177766897472 deprecation_wrapper.py:119] From /home/sf/data/EdgeML/tf/edgeml/utils.py:335: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.

W0809 11:18:12.201538 140177766897472 deprecation.py:323] From /home/sf/.local/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [28]:
k=1
x_test.shape

(16863, 5, 350, 8)

In [40]:
x_test[:64]

array([[[[ 8.01725443e-01, -3.94587280e-02,  4.08067890e-01, ...,
          -1.03118241e+00, -3.86102984e-02, -8.90117132e-01],
         [ 7.85085095e-01, -3.75467141e-02,  4.11676023e-01, ...,
          -1.02684695e+00, -4.58810468e-02, -8.87501712e-01],
         [ 8.09289566e-01, -6.43183283e-02,  4.21297533e-01, ...,
          -1.03204951e+00, -2.27468473e-02, -8.93479815e-01],
         ...,
         [ 7.51804399e-01,  2.26345970e-01,  3.34102320e-01, ...,
          -1.03172435e+00, -3.73617860e-02, -2.82218773e-01],
         [ 7.45753281e-01,  2.26345970e-01,  3.29892920e-01, ...,
          -1.02847275e+00, -3.98098495e-02, -2.77735196e-01],
         [ 7.51804399e-01,  2.41644361e-01,  3.28088943e-01, ...,
          -1.03150757e+00, -7.14877903e-02, -2.72130724e-01]],

        [[ 7.51804399e-01,  8.48387035e-02,  3.70784382e-01, ...,
          -1.03204951e+00, -3.24901398e-02, -8.54995776e-01],
         [ 7.41215168e-01,  9.43999129e-02,  3.72588538e-01, ...,
          -1.02847275e

In [42]:
%%time
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)

CPU times: user 4min 53s, sys: 23.4 s, total: 5min 16s
Wall time: 1min 2s


In [43]:
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

Accuracy at k = 1: 0.854771
